# Installs and imports

In [1]:
# install sktime
%pip install sktime
%pip install seaborn
%pip install joblib
%pip install numba

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime.classification.sklearn import RotationForest
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.ticker import MaxNLocator
import seaborn as sns
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
from sklearn.metrics import accuracy_score, classification_report

# Data Preprocessing

## Lior's 1st experiment

In [3]:
X_train = pd.read_csv("train_data/EDA_lior_1_results.csv")
X_train = X_train[120:]
X_train.head()

,Time rel [s],Day rel [d],Time rel [hh:mm:ss],Date abs [yyyy-mm-dd],Time abs [hh:mm:ss],ActivityClass [],Altitude [m],BodyPosition [],EdaArousalMean [],EdaSclMean [uS],EdaScrAmplitudesMean [uS],EdaScrCount [],EdaScrEnergiesMean [uSs],EdaScrHalfRecoveryTimesMean [s],EdaScrRiseTimesMean [s],InclinationDown [deg],InclinationForward [deg],InclinationRight [deg],MovementAcceleration [g],NonWearTime [],StepCount [steps],TempMean [],VerticalSpeed [m/s],temp [Grad Celsius],Marker,Segment id,Unnamed: 26
120,120,0,00:02:00,2023-07-11,18:47:49,2,107.486126,5,0.732626,6.388704,0.0,0,0.0,0.0,0.0,174.366907,90.053368,95.632838,0.009511,0,0,29.2,-0.000696,29.2,NaN,6,NaN
121,121,0,00:02:01,2023-07-11,18:47:50,2,107.511760,5,0.570570,6.687354,0.0,0,0.0,0.0,0.0,174.366907,90.053368,95.632838,0.007989,0,0,29.2,-0.001425,29.2,NaN,6,NaN
122,122,0,00:02:02,2023-07-11,18:47:51,2,107.557756,5,0.444360,6.989779,0.0,0,0.0,0.0,0.0,174.366907,90.053368,95.632838,0.003530,0,0,29.2,-0.001507,29.2,NaN,6,NaN
123,123,0,00:02:03,2023-07-11,18:47:52,2,107.629037,5,0.346068,7.145787,0.0,0,0.0,0.0,0.0,174.366907,90.053368,95.632838,0.003822,0,0,29.2,-0.000685,29.2,NaN,6,NaN
124,124,0,00:02:04,2023-07-11,18:47:53,2,107.723253,5,0.269518,7.146281,0.0,0,0.0,0.0,0.0,146.527444,75.823923,60.384142,0.003670,0,0,29.2,0.000843,29.2,NaN,6,NaN


In [4]:
Y_train = pd.read_csv("train_data/EDA_lior_1.csv")
Y_train = Y_train[6:]
Y_train

,Segment id,label
6,6,no
7,7,no
8,8,no
9,9,no
10,10,no
11,11,no
12,12,no
13,13,no
14,14,no
15,15,no


In [5]:
# Select the desired columns
selected_columns = ['Segment id', 'EdaArousalMean []']
df = pd.DataFrame(X_train)
X_data = df[selected_columns]
X_data

,Segment id,EdaArousalMean []
120,6,7.326261e-01
121,6,5.705698e-01
122,6,4.443602e-01
123,6,3.460681e-01
124,6,2.695181e-01
...,...,...
955,47,2.872370e-09
956,47,1.043517e-01
957,47,3.047191e-01
958,47,2.377635e-01


### Encoding the labels

In [7]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(Y_train.label)
encoded_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [8]:
Y_train["encoded_label"] = encoded_labels
Y_train

,Segment id,label,encoded_label
6,6,no,0
7,7,no,0
8,8,no,0
9,9,no,0
10,10,no,0
11,11,no,0
12,12,no,0
13,13,no,0
14,14,no,0
15,15,no,0


### Organaize data for training

In [9]:
# Merge the data and labels based on the 'series_id' column
merged_data = pd.merge(X_data, Y_train, on='Segment id')
merged_data

,Segment id,EdaArousalMean [],label,encoded_label
0,6,7.326261e-01,no,0
1,6,5.705698e-01,no,0
2,6,4.443602e-01,no,0
3,6,3.460681e-01,no,0
4,6,2.695181e-01,no,0
...,...,...,...,...
835,47,2.872370e-09,no,0
836,47,1.043517e-01,no,0
837,47,3.047191e-01,no,0
838,47,2.377635e-01,no,0


In [10]:
# Extract the measuremens columns as time series data
X_custom = merged_data.groupby('Segment id')['EdaArousalMean []'].apply(list).values
X_custom

array([list([0.732626147, 0.570569817, 0.44436022, 0.346068088, 0.269518098, 0.209900905, 0.177399313, 0.405005437, 0.750036689, 0.955810487, 0.744385956, 0.579728365, 0.451492905, 0.351623028, 0.273844289, 0.213270147, 0.166094957, 0.129354883, 0.100741684, 0.078457702]),
       list([0.06110292, 0.163063374, 1.135031661, 0.980113315, 0.764270813, 0.595214708, 0.463553681, 0.361015969, 0.28115952, 0.218967254, 0.170531869, 0.132810353, 0.103432807, 0.080553551, 0.062735169, 0.048858198, 0.038050803, 0.029633995, 0.023078979, 0.017973927]),
       list([0.013998108, 0.010901738, 0.008490282, 0.006612238, 0.005149616, 0.004010525, 0.0031234, 0.002432506, 0.001894438, 0.00147539, 0.001149035, 0.000894869, 0.000696925, 0.000542766, 0.000422706, 0.000329204, 0.000256384, 0.000199672, 0.000155505, 0.000121107]),
       list([9.43185e-05, 7.34553e-05, 5.72071e-05, 4.45529e-05, 3.46978e-05, 2.70227e-05, 2.10453e-05, 1.63901e-05, 1.27646e-05, 9.9411e-06, 7.74213e-06, 6.02958e-06, 4.69584e-06, 

In [11]:
# Extract the 'label' column as labels
Y_custom = Y_train["encoded_label"].values
Y_custom

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [12]:
# Convert X_custom to a 2D NumPy array
X_custom_np = np.vstack(X_custom)
X_custom_np

array([[7.32626147e-01, 5.70569817e-01, 4.44360220e-01, 3.46068088e-01,
        2.69518098e-01, 2.09900905e-01, 1.77399313e-01, 4.05005437e-01,
        7.50036689e-01, 9.55810487e-01, 7.44385956e-01, 5.79728365e-01,
        4.51492905e-01, 3.51623028e-01, 2.73844289e-01, 2.13270147e-01,
        1.66094957e-01, 1.29354883e-01, 1.00741684e-01, 7.84577020e-02],
       [6.11029200e-02, 1.63063374e-01, 1.13503166e+00, 9.80113315e-01,
        7.64270813e-01, 5.95214708e-01, 4.63553681e-01, 3.61015969e-01,
        2.81159520e-01, 2.18967254e-01, 1.70531869e-01, 1.32810353e-01,
        1.03432807e-01, 8.05535510e-02, 6.27351690e-02, 4.88581980e-02,
        3.80508030e-02, 2.96339950e-02, 2.30789790e-02, 1.79739270e-02],
       [1.39981080e-02, 1.09017380e-02, 8.49028200e-03, 6.61223800e-03,
        5.14961600e-03, 4.01052500e-03, 3.12340000e-03, 2.43250600e-03,
        1.89443800e-03, 1.47539000e-03, 1.14903500e-03, 8.94869000e-04,
        6.96925000e-04, 5.42766000e-04, 4.22706000e-04, 3.2920

## Lior's 2nd experiment

In [13]:
X_train1 = pd.read_csv("train_data/EDA_lior_2_results.csv")
X_train1 = X_train1[100:960]
X_train1.head()

,Time rel [s],Day rel [d],Time rel [hh:mm:ss],Date abs [yyyy-mm-dd],Time abs [hh:mm:ss],ActivityClass [],Altitude [m],BodyPosition [],EdaArousalMean [],EdaSclMean [uS],EdaScrAmplitudesMean [uS],EdaScrCount [],EdaScrEnergiesMean [uSs],EdaScrHalfRecoveryTimesMean [s],EdaScrRiseTimesMean [s],InclinationDown [deg],InclinationForward [deg],InclinationRight [deg],MovementAcceleration [g],NonWearTime [],StepCount [steps],TempMean [],VerticalSpeed [m/s],temp [Grad Celsius],Marker,Segment id,Unnamed: 26
100,100,0,00:01:40,2023-07-11,19:14:34,0.0,109.181945,0,0.219539,6.287830,0.0,0.0,0.0,0.0,0.0,11.669856,81.198953,97.602633,0.177910,0.0,0.0,29.4,-0.036872,29.4,NaN,5.0,NaN
101,101,0,00:01:41,2023-07-11,19:14:35,0.0,109.018117,0,0.172122,6.368778,0.0,0.0,0.0,0.0,0.0,11.669856,81.198953,97.602633,0.346009,0.0,0.0,29.4,-0.034635,29.4,NaN,5.0,NaN
102,102,0,00:01:42,2023-07-11,19:14:36,0.0,108.845850,0,0.134049,6.449124,0.0,0.0,0.0,0.0,0.0,11.669856,81.198953,97.602633,0.163379,0.0,0.0,29.4,-0.030700,29.4,NaN,5.0,NaN
103,103,0,00:01:43,2023-07-11,19:14:37,0.0,108.709690,0,0.104397,6.496478,0.0,0.0,0.0,0.0,0.0,11.669856,81.198953,97.602633,0.200110,0.0,0.0,29.5,-0.024966,29.5,NaN,5.0,NaN
104,104,0,00:01:44,2023-07-11,19:14:38,0.0,108.624707,0,0.081305,6.510152,0.0,0.0,0.0,0.0,0.0,26.914890,73.845746,110.920292,0.058747,0.0,0.0,29.5,-0.018029,29.5,NaN,5.0,NaN


In [14]:
Y_train1 = pd.read_csv("train_data/EDA_lior_2.csv")
Y_train1 = Y_train1[5:]
Y_train1

,Segment id,label
5,5,no
6,6,no
7,7,no
8,8,no
9,9,no
10,10,no
11,11,no
12,12,no
13,13,no
14,14,yes


In [15]:
# Select the desired columns
selected_columns1 = ['Segment id', 'EdaArousalMean []']
df1 = pd.DataFrame(X_train1)
X_data1 = df1[selected_columns1]
X_data1

,Segment id,EdaArousalMean []
100,5.0,0.219539
101,5.0,0.172122
102,5.0,0.134049
103,5.0,0.104397
104,5.0,0.081305
...,...,...
955,47.0,0.002364
956,47.0,0.001841
957,47.0,0.001434
958,47.0,0.001117


### Encoding the labels

In [16]:
encoded_labels1 = label_encoder.fit_transform(Y_train1.label)
encoded_labels1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
Y_train1["encoded_label"] = encoded_labels1
Y_train1

,Segment id,label,encoded_label
5,5,no,0
6,6,no,0
7,7,no,0
8,8,no,0
9,9,no,0
10,10,no,0
11,11,no,0
12,12,no,0
13,13,no,0
14,14,yes,1


### Organaize data for training

In [18]:
# Merge the data and labels based on the 'series_id' column
merged_data1 = pd.merge(X_data1, Y_train1, on='Segment id')
merged_data1

,Segment id,EdaArousalMean [],label,encoded_label
0,5.0,0.219539,no,0
1,5.0,0.172122,no,0
2,5.0,0.134049,no,0
3,5.0,0.104397,no,0
4,5.0,0.081305,no,0
...,...,...,...,...
855,47.0,0.002364,no,0
856,47.0,0.001841,no,0
857,47.0,0.001434,no,0
858,47.0,0.001117,no,0


In [19]:
# Extract the measuremens columns as time series data
X_custom1 = merged_data1.groupby('Segment id')['EdaArousalMean []'].apply(list).values
X_custom1

array([list([0.219538958, 0.172122288, 0.134048973, 0.104397445, 0.081304812, 0.063320251, 0.049313861, 0.038405674, 0.029910369, 0.023294219, 0.018141556, 0.014128658, 0.01100341, 0.008569464, 0.006673905, 0.005197643, 0.004047928, 0.00315253, 0.002455193, 0.001912106]),
       list([0.00148915, 0.001159751, 0.000903215, 0.000703424, 0.000547828, 0.000426648, 0.000332274, 0.000258775, 0.000201534, 0.000156955, 0.000122237, 9.51981e-05, 7.41404e-05, 5.77406e-05, 0.37831646, 0.352290196, 0.27436388, 0.213674805, 0.166410105, 0.12960032]),
       list([0.100932831, 0.078606568, 0.061218857, 0.047677293, 0.037131113, 0.02891774, 0.022521159, 0.017539496, 0.013659773, 0.010638242, 0.008285071, 0.00645242, 0.00502515, 0.003913591, 0.003047907, 0.002373713, 0.001848649, 0.00143973, 0.001121262, 0.00087324]),
       list([0.00068008, 0.000529647, 0.000412489, 0.000321247, 0.000250187, 0.000194846, 0.000151746, 0.00011818, 9.20388e-05, 7.16799e-05, 5.58244e-05, 4.34761e-05, 3.38592e-05, 2.6369

In [22]:
X_custom 

array([list([0.732626147, 0.570569817, 0.44436022, 0.346068088, 0.269518098, 0.209900905, 0.177399313, 0.405005437, 0.750036689, 0.955810487, 0.744385956, 0.579728365, 0.451492905, 0.351623028, 0.273844289, 0.213270147, 0.166094957, 0.129354883, 0.100741684, 0.078457702]),
       list([0.06110292, 0.163063374, 1.135031661, 0.980113315, 0.764270813, 0.595214708, 0.463553681, 0.361015969, 0.28115952, 0.218967254, 0.170531869, 0.132810353, 0.103432807, 0.080553551, 0.062735169, 0.048858198, 0.038050803, 0.029633995, 0.023078979, 0.017973927]),
       list([0.013998108, 0.010901738, 0.008490282, 0.006612238, 0.005149616, 0.004010525, 0.0031234, 0.002432506, 0.001894438, 0.00147539, 0.001149035, 0.000894869, 0.000696925, 0.000542766, 0.000422706, 0.000329204, 0.000256384, 0.000199672, 0.000155505, 0.000121107]),
       list([9.43185e-05, 7.34553e-05, 5.72071e-05, 4.45529e-05, 3.46978e-05, 2.70227e-05, 2.10453e-05, 1.63901e-05, 1.27646e-05, 9.9411e-06, 7.74213e-06, 6.02958e-06, 4.69584e-06, 

In [20]:
merged_X1 = np.concatenate((X_custom, X_custom1), axis=0)
merged_X1

array([list([0.732626147, 0.570569817, 0.44436022, 0.346068088, 0.269518098, 0.209900905, 0.177399313, 0.405005437, 0.750036689, 0.955810487, 0.744385956, 0.579728365, 0.451492905, 0.351623028, 0.273844289, 0.213270147, 0.166094957, 0.129354883, 0.100741684, 0.078457702]),
       list([0.06110292, 0.163063374, 1.135031661, 0.980113315, 0.764270813, 0.595214708, 0.463553681, 0.361015969, 0.28115952, 0.218967254, 0.170531869, 0.132810353, 0.103432807, 0.080553551, 0.062735169, 0.048858198, 0.038050803, 0.029633995, 0.023078979, 0.017973927]),
       list([0.013998108, 0.010901738, 0.008490282, 0.006612238, 0.005149616, 0.004010525, 0.0031234, 0.002432506, 0.001894438, 0.00147539, 0.001149035, 0.000894869, 0.000696925, 0.000542766, 0.000422706, 0.000329204, 0.000256384, 0.000199672, 0.000155505, 0.000121107]),
       list([9.43185e-05, 7.34553e-05, 5.72071e-05, 4.45529e-05, 3.46978e-05, 2.70227e-05, 2.10453e-05, 1.63901e-05, 1.27646e-05, 9.9411e-06, 7.74213e-06, 6.02958e-06, 4.69584e-06, 

In [21]:
# Extract the 'label' column as labels
Y_custom1 = Y_train1["encoded_label"].values
Y_custom1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [23]:
Y_custom

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [24]:
merged_Y1 = np.concatenate((Y_custom, Y_custom1), axis=0)
merged_Y1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [25]:
# Convert X_custom to a 2D NumPy array
X_custom_np1 = np.vstack(merged_X1)
X_custom_np1

array([[7.32626147e-01, 5.70569817e-01, 4.44360220e-01, ...,
        1.29354883e-01, 1.00741684e-01, 7.84577020e-02],
       [6.11029200e-02, 1.63063374e-01, 1.13503166e+00, ...,
        2.96339950e-02, 2.30789790e-02, 1.79739270e-02],
       [1.39981080e-02, 1.09017380e-02, 8.49028200e-03, ...,
        1.99672000e-04, 1.55505000e-04, 1.21107000e-04],
       ...,
       [9.67693000e-25, 7.53640000e-25, 5.86936000e-25, ...,
        1.38034000e-26, 1.07501000e-26, 8.37219000e-27],
       [6.52027000e-27, 5.07799000e-27, 3.95474000e-27, ...,
        2.00191845e-01, 1.65717189e-01, 1.29060676e-01],
       [1.00512556e-01, 7.82792570e-02, 6.09639470e-02, ...,
        1.43373500e-03, 1.11659400e-03, 8.69604000e-04]])

## Vlad's 1st experiment

In [26]:
X_train2 = pd.read_csv("train_data/EDA_vlad_1_results.csv")
# Take segments id 19 to 58
X_train2 = X_train2[380:1180]
X_train2

,Time rel [s],Day rel [d],Time rel [hh:mm:ss],Date abs [yyyy-mm-dd],Time abs [hh:mm:ss],ActivityClass [],Altitude [m],BodyPosition [],EdaArousalMean [],EdaSclMean [uS],EdaScrAmplitudesMean [uS],EdaScrCount [],EdaScrEnergiesMean [uSs],EdaScrHalfRecoveryTimesMean [s],EdaScrRiseTimesMean [s],InclinationDown [deg],InclinationForward [deg],InclinationRight [deg],MovementAcceleration [g],NonWearTime [],StepCount [steps],TempMean [],VerticalSpeed [m/s],temp [Grad Celsius],Markers,Segment id,Unnamed: 26
380,380,0,00:06:20,2023-07-11,16:56:57,2.0,97.714295,5.0,2.315570e-02,8.349160,0.0,0.0,0.0,0.0,0.0,120.722810,63.582414,42.645281,0.004673,0.0,0.0,29.0,-0.006225,29.0,NaN,19.0,NaN
381,381,0,00:06:21,2023-07-11,16:56:58,2.0,97.702249,5.0,1.803368e-02,8.298571,0.0,0.0,0.0,0.0,0.0,120.722810,63.582414,42.645281,0.004744,0.0,0.0,29.0,-0.006348,29.0,NaN,19.0,NaN
382,382,0,00:06:22,2023-07-11,16:56:59,2.0,97.684459,5.0,1.404464e-02,8.253473,0.0,0.0,0.0,0.0,0.0,120.722810,63.582414,42.645281,0.003056,0.0,0.0,29.0,-0.006390,29.0,NaN,19.0,NaN
383,383,0,00:06:23,2023-07-11,16:57:00,2.0,97.664046,5.0,1.093798e-02,8.215360,0.0,0.0,0.0,0.0,0.0,120.722810,63.582414,42.645281,0.003090,0.0,0.0,29.0,-0.006052,29.0,NaN,19.0,NaN
384,384,0,00:06:24,2023-07-11,16:57:01,2.0,97.645544,5.0,8.518505e-03,8.187131,0.0,0.0,0.0,0.0,0.0,120.881924,64.061208,42.405239,0.005561,0.0,0.0,29.0,-0.005315,29.0,NaN,19.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,1175,0,00:19:35,2023-07-11,17:10:12,1.0,99.016731,1.0,7.216300e-16,4.513154,0.0,0.0,0.0,0.0,0.0,72.234272,18.252975,85.944887,0.004095,0.0,0.0,29.4,0.035207,29.4,NaN,58.0,NaN
1176,1176,0,00:19:36,2023-07-11,17:10:13,7.0,99.045522,1.0,5.620060e-16,4.512908,0.0,0.0,0.0,0.0,0.0,71.527849,18.988728,85.753281,0.003471,0.0,0.0,29.4,0.035628,29.4,NaN,58.0,NaN
1177,1177,0,00:19:37,2023-07-11,17:10:14,7.0,99.092540,1.0,4.376910e-16,4.512259,0.0,0.0,0.0,0.0,0.0,71.527849,18.988728,85.753281,0.003090,0.0,0.0,29.4,0.035448,29.4,NaN,58.0,NaN
1178,1178,0,00:19:38,2023-07-11,17:10:15,7.0,99.157696,1.0,3.408740e-16,4.513837,0.0,0.0,0.0,0.0,0.0,71.527849,18.988728,85.753281,0.003929,0.0,0.0,29.4,0.035074,29.4,NaN,58.0,NaN


In [27]:
Y_train2 = pd.read_csv("train_data/EDA_vlad_1.csv")
# Take segments id 19 to 58
Y_train2 = Y_train2[19:59]
Y_train2

,Segment id,label
19,19,no
20,20,no
21,21,no
22,22,no
23,23,no
24,24,no
25,25,no
26,26,no
27,27,no
28,28,no


In [28]:
# Select the desired columns
selected_columns2 = ['Segment id', 'EdaArousalMean []']
df2 = pd.DataFrame(X_train2)
X_data2 = df2[selected_columns1]
X_data2

,Segment id,EdaArousalMean []
380,19.0,2.315570e-02
381,19.0,1.803368e-02
382,19.0,1.404464e-02
383,19.0,1.093798e-02
384,19.0,8.518505e-03
...,...,...
1175,58.0,7.216300e-16
1176,58.0,5.620060e-16
1177,58.0,4.376910e-16
1178,58.0,3.408740e-16


### Encoding the labels

In [29]:
encoded_labels2 = label_encoder.fit_transform(Y_train2.label)
encoded_labels2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [30]:
Y_train2["encoded_label"] = encoded_labels2
Y_train2

,Segment id,label,encoded_label
19,19,no,0
20,20,no,0
21,21,no,0
22,22,no,0
23,23,no,0
24,24,no,0
25,25,no,0
26,26,no,0
27,27,no,0
28,28,no,0


### Organizing data for training

In [31]:
# Merge the data and labels based on the 'series_id' column
merged_data2 = pd.merge(X_data2, Y_train2, on='Segment id')
merged_data2

,Segment id,EdaArousalMean [],label,encoded_label
0,19.0,2.315570e-02,no,0
1,19.0,1.803368e-02,no,0
2,19.0,1.404464e-02,no,0
3,19.0,1.093798e-02,no,0
4,19.0,8.518505e-03,no,0
...,...,...,...,...
795,58.0,7.216300e-16,no,0
796,58.0,5.620060e-16,no,0
797,58.0,4.376910e-16,no,0
798,58.0,3.408740e-16,no,0


In [32]:
# Extract the measuremens columns as time series data
X_custom2 = merged_data2.groupby('Segment id')['EdaArousalMean []'].apply(list).values
X_custom2

array([list([0.023155699, 0.018033676, 0.014044641, 0.010937978, 0.008518505, 0.006634219, 0.005166735, 0.004023857, 0.003133783, 0.002440593, 0.001900735, 0.001480294, 0.001152854, 0.000897844, 0.000699242, 0.00054457, 0.000424111, 0.000330298, 0.000257237, 0.000200336]),
       list([0.000156022, 0.00012151, 9.4632e-05, 7.36995e-05, 5.73972e-05, 4.4701e-05, 3.48132e-05, 2.71125e-05, 2.11153e-05, 1.64446e-05, 1.28071e-05, 9.97414e-06, 7.76787e-06, 6.04962e-06, 4.71145e-06, 3.66928e-06, 2.85764e-06, 2.22553e-06, 1.73325e-06, 1.34985e-06]),
       list([1.05127e-06, 8.18728e-07, 6.37626e-07, 4.96583e-07, 3.8674e-07, 3.01193e-07, 2.34569e-07, 1.82683e-07, 1.42274e-07, 1.10803e-07, 8.62933e-08, 6.72053e-08, 5.23395e-08, 0.015763245, 0.303056237, 0.245865009, 0.191479862, 0.149124666, 0.116138407, 0.090448682]),
       list([0.070441505, 0.054859899, 0.042724932, 0.033274211, 0.113932717, 0.128482878, 0.100062566, 0.077928805, 0.146857909, 0.166194591, 0.129432478, 0.150949921, 0.158337828

In [33]:
merged_X1

array([list([0.732626147, 0.570569817, 0.44436022, 0.346068088, 0.269518098, 0.209900905, 0.177399313, 0.405005437, 0.750036689, 0.955810487, 0.744385956, 0.579728365, 0.451492905, 0.351623028, 0.273844289, 0.213270147, 0.166094957, 0.129354883, 0.100741684, 0.078457702]),
       list([0.06110292, 0.163063374, 1.135031661, 0.980113315, 0.764270813, 0.595214708, 0.463553681, 0.361015969, 0.28115952, 0.218967254, 0.170531869, 0.132810353, 0.103432807, 0.080553551, 0.062735169, 0.048858198, 0.038050803, 0.029633995, 0.023078979, 0.017973927]),
       list([0.013998108, 0.010901738, 0.008490282, 0.006612238, 0.005149616, 0.004010525, 0.0031234, 0.002432506, 0.001894438, 0.00147539, 0.001149035, 0.000894869, 0.000696925, 0.000542766, 0.000422706, 0.000329204, 0.000256384, 0.000199672, 0.000155505, 0.000121107]),
       list([9.43185e-05, 7.34553e-05, 5.72071e-05, 4.45529e-05, 3.46978e-05, 2.70227e-05, 2.10453e-05, 1.63901e-05, 1.27646e-05, 9.9411e-06, 7.74213e-06, 6.02958e-06, 4.69584e-06, 

In [34]:
merged_X2 = np.concatenate((merged_X1, X_custom2), axis=0)
merged_X2

array([list([0.732626147, 0.570569817, 0.44436022, 0.346068088, 0.269518098, 0.209900905, 0.177399313, 0.405005437, 0.750036689, 0.955810487, 0.744385956, 0.579728365, 0.451492905, 0.351623028, 0.273844289, 0.213270147, 0.166094957, 0.129354883, 0.100741684, 0.078457702]),
       list([0.06110292, 0.163063374, 1.135031661, 0.980113315, 0.764270813, 0.595214708, 0.463553681, 0.361015969, 0.28115952, 0.218967254, 0.170531869, 0.132810353, 0.103432807, 0.080553551, 0.062735169, 0.048858198, 0.038050803, 0.029633995, 0.023078979, 0.017973927]),
       list([0.013998108, 0.010901738, 0.008490282, 0.006612238, 0.005149616, 0.004010525, 0.0031234, 0.002432506, 0.001894438, 0.00147539, 0.001149035, 0.000894869, 0.000696925, 0.000542766, 0.000422706, 0.000329204, 0.000256384, 0.000199672, 0.000155505, 0.000121107]),
       list([9.43185e-05, 7.34553e-05, 5.72071e-05, 4.45529e-05, 3.46978e-05, 2.70227e-05, 2.10453e-05, 1.63901e-05, 1.27646e-05, 9.9411e-06, 7.74213e-06, 6.02958e-06, 4.69584e-06, 

In [35]:
# Extract the 'label' column as labels
Y_custom2 = Y_train2["encoded_label"].values
Y_custom2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [36]:
merged_Y1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [37]:
merged_Y2 = np.concatenate((merged_Y1, Y_custom2), axis=0)
merged_Y2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [38]:
# Convert X_custom to a 2D NumPy array
X_custom_np2 = np.vstack(merged_X2)
X_custom_np2

array([[7.32626147e-01, 5.70569817e-01, 4.44360220e-01, ...,
        1.29354883e-01, 1.00741684e-01, 7.84577020e-02],
       [6.11029200e-02, 1.63063374e-01, 1.13503166e+00, ...,
        2.96339950e-02, 2.30789790e-02, 1.79739270e-02],
       [1.39981080e-02, 1.09017380e-02, 8.49028200e-03, ...,
        1.99672000e-04, 1.55505000e-04, 1.21107000e-04],
       ...,
       [6.75871000e-10, 5.26369000e-10, 4.09936000e-10, ...,
        9.64078000e-12, 7.50824000e-12, 5.84743000e-12],
       [4.55398000e-12, 3.54664000e-12, 2.76213000e-12, ...,
        6.49590000e-14, 5.05902000e-14, 3.93997000e-14],
       [3.06845000e-14, 2.38971000e-14, 1.86111000e-14, ...,
        4.37691000e-16, 3.40874000e-16, 2.65473000e-16]])

# Train and save model

## Lior's experiment 1 data

In [64]:
clf = ShapeletTransformClassifier(estimator=RotationForest())
clf.fit(X_custom_np, Y_custom)

ShapeletTransformClassifier(estimator=RotationForest())

In [65]:
# Specify the directory and filename where you want to save the model
save_directory = './model/'
model_filename = 'eda_shapelet_model.pkl'

# Combine the directory and filename to create the full file path
full_model_path = os.path.join(save_directory, model_filename)

# Save the trained model to the specified directory
joblib.dump(clf, full_model_path)

['./model/eda_shapelet_model.pkl']

## Lior's experiment 2 data

In [66]:
clf1 = ShapeletTransformClassifier(estimator=RotationForest())
clf1.fit(X_custom_np1, merged_Y1)


ShapeletTransformClassifier(estimator=RotationForest())

In [67]:
# Specify the directory and filename where you want to save the model
save_directory = './model/'
model_filename = 'eda_shapelet_model1.pkl'

# Combine the directory and filename to create the full file path
full_model_path = os.path.join(save_directory, model_filename)

# Save the trained model to the specified directory
joblib.dump(clf1, full_model_path)

['./model/eda_shapelet_model1.pkl']

## Vlad experiment 1 data

In [68]:
clf2 = ShapeletTransformClassifier(estimator=RotationForest())
clf2.fit(X_custom_np2, merged_Y2)

ShapeletTransformClassifier(estimator=RotationForest())

In [69]:
# Specify the directory and filename where you want to save the model
save_directory = './model/'
model_filename = 'eda_shapelet_model2.pkl'

# Combine the directory and filename to create the full file path
full_model_path = os.path.join(save_directory, model_filename)

# Save the trained model to the specified directory
joblib.dump(clf2, full_model_path)

['./model/eda_shapelet_model2.pkl']

# Load and test model

## Organaize data for testing

In [70]:
test_data = pd.read_csv("test_data/EDA_vlad_2_results.csv")
test_data = test_data[60:880]
test_data

,Time rel [s],Day rel [d],Time rel [hh:mm:ss],Date abs [yyyy-mm-dd],Time abs [hh:mm:ss],ActivityClass [],Altitude [m],BodyPosition [],EdaArousalMean [],EdaSclMean [uS],EdaScrAmplitudesMean [uS],EdaScrCount [],EdaScrEnergiesMean [uSs],EdaScrHalfRecoveryTimesMean [s],EdaScrRiseTimesMean [s],InclinationDown [deg],InclinationForward [deg],InclinationRight [deg],MovementAcceleration [g],NonWearTime [],StepCount [steps],TempMean [],VerticalSpeed [m/s],temp [Grad Celsius],Marker,Segment id
60,60,0,00:01:00,2023-07-11,17:26:01,2,100.536476,5,0.427687,9.062784,0.00000,0,0.000000,0.0,0.00000,127.075971,87.766423,37.166398,0.227796,0,1,27.7,-0.020075,27.7,NaN,3
61,61,0,00:01:01,2023-07-11,17:26:02,2,100.509342,5,0.333083,9.143783,0.00000,0,0.000000,0.0,0.00000,127.075971,87.766423,37.166398,0.278153,0,2,27.7,-0.017381,27.7,NaN,3
62,62,0,00:01:02,2023-07-11,17:26:03,2,100.458276,5,0.259405,9.148984,0.00000,0,0.000000,0.0,0.00000,127.075971,87.766423,37.166398,0.210692,0,0,27.7,-0.013896,27.7,Mark - Start,3
63,63,0,00:01:03,2023-07-11,17:26:04,2,100.394854,5,0.202025,9.050531,0.00000,0,0.000000,0.0,0.00000,127.075971,87.766423,37.166398,0.304300,0,1,27.7,-0.010250,27.7,NaN,3
64,64,0,00:01:04,2023-07-11,17:26:05,2,100.343395,5,0.497035,8.671568,0.79298,1,0.284977,0.0,0.71875,127.320149,107.638195,42.669686,0.224496,0,0,27.7,-0.006618,27.7,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,875,0,00:14:35,2023-07-11,17:39:36,1,102.786522,2,0.001763,7.564490,0.00000,0,0.000000,0.0,0.00000,97.863848,112.253360,23.744604,0.010406,0,0,29.3,0.011734,29.3,NaN,43
876,876,0,00:14:36,2023-07-11,17:39:37,1,102.772806,2,0.001373,7.541065,0.00000,0,0.000000,0.0,0.00000,97.678370,111.985071,23.421598,0.008280,0,0,29.3,0.012667,29.3,NaN,43
877,877,0,00:14:37,2023-07-11,17:39:38,1,102.765797,2,0.001069,7.527106,0.00000,0,0.000000,0.0,0.00000,97.678370,111.985071,23.421598,0.006656,0,0,29.3,0.013976,29.3,NaN,43
878,878,0,00:14:38,2023-07-11,17:39:39,1,102.770840,2,0.000833,7.525796,0.00000,0,0.000000,0.0,0.00000,97.678370,111.985071,23.421598,0.007316,0,0,29.3,0.015057,29.3,NaN,43


In [71]:
Y_test = pd.read_csv("test_data/EDA_vlad_2.csv")
Y_test = Y_test[3:44]
Y_test

,Segment id,label
3,3,no
4,4,no
5,5,no
6,6,no
7,7,no
8,8,no
9,9,no
10,10,no
11,11,no
12,12,no


In [72]:
# Get the real labels of the testing data
test_encoded_labels = label_encoder.fit_transform(Y_test.label)
test_encoded_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [73]:
Y_test["encoded_label"] = test_encoded_labels
Y_test

,Segment id,label,encoded_label
3,3,no,0
4,4,no,0
5,5,no,0
6,6,no,0
7,7,no,0
8,8,no,0
9,9,no,0
10,10,no,0
11,11,no,0
12,12,no,0


In [74]:
# Extract the measuremens columns as time series data
X_test = test_data.groupby('Segment id')['EdaArousalMean []'].apply(list).values
X_test_np = np.vstack(X_test)
X_test_np

array([[4.27687084e-01, 3.33083036e-01, 2.59405329e-01, 2.02025074e-01,
        4.97034597e-01, 6.81649580e-01, 5.30869227e-01, 4.13441369e-01,
        3.21988462e-01, 2.50764867e-01, 1.95295874e-01, 1.52096580e-01,
        1.18452936e-01, 9.22512390e-02, 7.18453370e-02, 5.59532050e-02,
        4.35764000e-02, 3.39373340e-02, 2.64304220e-02, 2.05840340e-02],
       [1.60308620e-02, 1.24848480e-02, 9.72320900e-03, 7.57244300e-03,
        5.89742400e-03, 4.59291900e-03, 3.57696900e-03, 2.78574600e-03,
        2.16954100e-03, 1.68964000e-03, 1.31589300e-03, 1.02481900e-03,
        7.98130000e-04, 6.21584000e-04, 4.84090000e-04, 3.77010000e-04,
        2.93615000e-04, 2.28668000e-04, 1.78087000e-04, 1.38694000e-04],
       [1.08015000e-04, 8.41222000e-05, 6.55145000e-05, 5.10227000e-05,
        3.97365000e-05, 3.09468000e-05, 2.41014000e-05, 1.87702000e-05,
        8.62759539e-01, 1.17398173e+00, 9.14297890e-01, 7.12055913e-01,
        5.54549703e-01, 4.31883743e-01, 3.36351397e-01, 4.0019

## Test Lior exp 1 model

In [75]:
# Load the model that is trained only on Lior's 1st experiment data
loaded_clf = joblib.load("./model/eda_shapelet_model.pkl")

In [76]:
# Predict labels for the test data
y_pred = loaded_clf.predict(X_test_np)
# Print predictions
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [77]:
# Calculate accuracy of the model on Vlad's 2nd experiment data
accuracy = accuracy_score(test_encoded_labels, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
report = classification_report(test_encoded_labels, y_pred)
print(report)

Accuracy: 87.80%
              precision    recall  f1-score   support

           0       0.95      0.92      0.94        39
           1       0.00      0.00      0.00         2

    accuracy                           0.88        41
   macro avg       0.47      0.46      0.47        41
weighted avg       0.90      0.88      0.89        41



## Test Lior exp 1+2 model

In [78]:
# Load the model that is trained on Lior's 1st and 2nd experiments data
loaded_clf1 = joblib.load("./model/eda_shapelet_model1.pkl")

In [79]:
# Predict labels for the test data
y_pred1 = loaded_clf1.predict(X_test_np)
# Print predictions
y_pred1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0])

In [80]:
# Calculate accuracy of the model on Vlad's 2nd experiment data
accuracy1 = accuracy_score(test_encoded_labels, y_pred1)
print(f"Accuracy: {accuracy1 * 100:.2f}%")
report = classification_report(test_encoded_labels, y_pred1)
print(report)

Accuracy: 95.12%
              precision    recall  f1-score   support

           0       1.00      0.95      0.97        39
           1       0.50      1.00      0.67         2

    accuracy                           0.95        41
   macro avg       0.75      0.97      0.82        41
weighted avg       0.98      0.95      0.96        41



## Test Lior exp 1+2 and Vlad exp 1 model

In [81]:
# Load the model that is trained on Lior's 1st and 2nd experiments and Vlad's 1s experiment data
loaded_clf2 = joblib.load("./model/eda_shapelet_model2.pkl")

In [82]:
# Predict labels for the test data
y_pred2 = loaded_clf2.predict(X_test_np)
# Print predictions
y_pred2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [83]:
# Calculate accuracy of the model on Vlad's 2nd experiment data
accuracy2 = accuracy_score(test_encoded_labels, y_pred2)
print(f"Accuracy: {accuracy2 * 100:.2f}%")
report = classification_report(test_encoded_labels, y_pred2)
print(report)

Accuracy: 87.80%
              precision    recall  f1-score   support

           0       0.95      0.92      0.94        39
           1       0.00      0.00      0.00         2

    accuracy                           0.88        41
   macro avg       0.47      0.46      0.47        41
weighted avg       0.90      0.88      0.89        41

